# Advent of Code 2024

> If debugging is the process of removing software bugs, then programming must be the process of putting them in.

-- Edsger W. Dijkstra

## Imports and definitions

In [1]:
from urllib import request
from collections import Counter

def aocin(day):
    try:
        with open(f'input/{day}') as f:
            return f.read().strip()
    except FileNotFoundError:
        r = request.Request(f'https://adventofcode.com/2024/day/{day}/input')
        r.add_header('Cookie', open('../.aoccookie').read().strip())
        r.add_header('User-Agent', 'github.com/edoannunziata/jardin')
        with open(f'input/{day}', 'bw') as f:
            f.write(request.urlopen(r).read())
        with open(f'input/{day}') as f:
            return f.read().strip()

## [Day 1: Historian Hysteria](https://adventofcode.com/2024/day/1)

In [2]:
a, b = [list(t) for t in zip(*[[*map(int, x.split())] for x in aocin(1).split('\n')])]
na, nb = Counter(a), Counter(b)

A = sum(abs(u-v) for u, v in zip(sorted(a), sorted(b)))
assert A == 1388114

A = sum(u * nu * nb[u] for u, nu in na.items())
assert A == 23529853

## [Day 2: Red-Nosed Reports](https://adventofcode.com/2024/day/2)

In [3]:
reports = [[*map(int, l.split())] for l in aocin(2).split('\n')]

def is_safe(l, allowed, tol=0):
    def _f(l, tol):
        if tol < 0: return False
        if len(l) < 2: return True
        a, b, *rest = l
        return (
            ((a is None or a-b in allowed) and _f([b] + rest, tol))
            or _f([a] + rest, tol-1)
        )
    return _f([None] + l, tol)


A = sum(is_safe(l, {1, 2, 3}) or is_safe(l, {-1, -2, -3}) for l in reports)
assert A == 369

A = sum(is_safe(l, {1, 2, 3}, tol=1) or is_safe(l, {-1, -2, -3}, tol=1) for l in reports)
assert A == 428